# Training a Neural Network using Augmentor and Keras

In this notebook, we will train a simple convolutional neural network on the MNIST dataset using Augmentor to augment images on the fly using a generator.

## Import Required Libraries

We start by making a number of imports:

In [8]:
import Augmentor

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Define a Convolutional Neural Network

Once the libraries have been imported, we define a small convolutional neural network. See the Keras documentation for details of this network: <https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py> 

It is a three layer deep neural network, consisting of 2 convolutional layers and a fully connected layer:

In [9]:
num_classes = 16
input_shape = (300, 300, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Once a network has been defined, you can compile it so that the model is ready to be trained with data:

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

You can view a summary of the network using the `summary()` function:

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 296, 296, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 148, 148, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1401856)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               179437696 
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

After the MNIST data has downloaded, we can instantiate a `Pipeline` object in the `training` directory to add the images to the current pipeline:

In [18]:
#p = Augmentor.Pipeline("versi0")
# Passing the path of the image directory 
p = Augmentor.Pipeline(source_directory="versi0/train",
                      output_directory="versi0_basicaugment/train")

Initialised with 310 image(s) found.
Output directory set to versi0/train/versi0_basicaugment/train.

## Add Operations to the Pipeline

Now that a pipeline object `p` has been created, we can add operations to the pipeline. Below we add several simple  operations:

In [19]:
# Passing the path of the image directory 

  
# Defining augmentation parameters and generating 10 samples 
p.flip_left_right(probability=0.4) 
p.flip_top_bottom(probability=0.8)
p.rotate(probability=0.5, max_left_rotation=5, max_right_rotation=10)
p.skew(0.4, 0.5) 
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5) 
p.sample(620)

Processing <PIL.Image.Image image mode=RGB size=500x333 at 0x166212250>: 100%|██████████| 620/620 [00:17<00:00, 35.72 Samples/s]                   


You can view the status of pipeline using the `status()` function, which shows information regarding the number of classes in the pipeline, the number of images, and what operations have been added to the pipeline:

In [5]:
p.status()
#p.remove_operation()


Operations: 5
	0: Flip (probability=0.4 top_bottom_left_right=LEFT_RIGHT )
	1: Flip (probability=0.8 top_bottom_left_right=TOP_BOTTOM )
	2: RotateRange (probability=0.5 max_left_rotation=-5 max_right_rotation=10 )
	3: Skew (probability=0.4 skew_type=RANDOM magnitude=0.5 )
	4: Zoom (probability=0.2 min_factor=1.1 max_factor=1.5 )
Images: 310
Classes: 16
	Class index: 0 Class label: apoderus_javanicus 
	Class index: 1 Class label: aulacaspis_tubercularis 
	Class index: 2 Class label: ceroplastes_rubens 
	Class index: 3 Class label: cisaberoptus_kenyae 
	Class index: 4 Class label: dappula_tertia 
	Class index: 5 Class label: dialeuropora_decempuncta 
	Class index: 6 Class label: erosomyia_sp 
	Class index: 7 Class label: icerya_seychellarum 
	Class index: 8 Class label: ischnaspis_longirostris 
	Class index: 9 Class label: mictis_longicornis 
	Class index: 10 Class label: neomelicharia_sparsa 
	Class index: 11 Class label: normal 
	Class index: 12 Class label: orthaga_euadrusalis 
	Class

## Creating a Generator

A generator will create images indefinitely, and we can use this generator as input into the model created above. The generator is created with a user-defined batch size, which we define here in a variable named `batch_size`. This is used later to define number of steps per epoch, so it is best to keep it stored as a variable.

In [6]:
batch_size = 2
g = p.keras_generator(batch_size=batch_size)
#g = p.image_generator()
print(g)

<generator object Pipeline.keras_generator at 0x166732750>


The generator can now be used to created augmented data. In Python, generators are invoked using the `next()` function - the Augmentor generators will return images indefinitely, and so `next()` can be called as often as required. 

You can view the output of generator manually:

In [7]:
images, labels = next(g)

/opt/anaconda3/envs/MangoLeaf/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


ValueError: setting an array element with a sequence.

Images, and their labels, are returned in batches of the size defined above by `batch_size`. The `image_batch` variable is a tuple, containing the augmentented images and their corresponding labels.

To see the label of the first image returned by the generator you can use the array's index:

In [ ]:
print(labels[0])
print(images[0])
print(images[0].shape)

Or preview the images using Matplotlib (the image should be a 5, according to the label information above):

In [ ]:
plt.imshow(images[0].reshape(333, 500, 3), cmap="Greys");

## Train the Network

We train the network by passing the generator, `g`, to the model's fit function. In Keras, if a generator is used we used the `fit_generator()` function as opposed to the standard `fit()` function. Also, the steps per epoch should roughly equal the total number of images in your dataset divided by the `batch_size`.

Training the network over 5 epochs, we get the following output:

In [ ]:
h = model.fit_generator(g, steps_per_epoch=len(p.augmentor_images)/batch_size, epochs=2, verbose=1)

## Summary

Using Augmentor with Keras means only that you need to create a generator when you are finished creating your pipeline. This has the advantage that no images need to be saved to disk and are augmented on the fly.